### Overview of the Jupyter Notebook and Utils Module

I have developed several **Jupyter Notebooks** that demonstrate key functionalities related to **FHIR and VRS** schema interactions, including:  

- Creating a **FHIR Allele**  
- Creating a **FHIR Sequence**  
- Constructing a **MolecularDefinition** resource  
- Performing **bidirectional translation** between **VRS and FHIR**  

However, these notebooks require users to have a **working knowledge of Jupyter Notebooks, Python, and the schema of both standards (FHIR & VRS)**.  
This notebook simplifies our workflow by reducing the need for extensive background knowledge. It provides a structured approach for efficiently generating and translating data between FHIR and VRS.

### Introducing the Builder Module: `allele_builder.py`

To streamline the creation of Allele objects, we developed the **`allele_builder.py`** module, located in the **builders directory**. This module simplifies the process by allowing users to generate **FHIR Allele** and **VRS Allele** objects with only **five key attributes**.  

### Functions in `allele_builder.py`  

#### **`build_fhir_allele()` – Generates a FHIR Allele**  

This function constructs a **FHIR Allele** using the following attributes:  

- `context_sequence_id` (**str**): Accession number of the reference sequence. Supported prefixes include: ("NC_", "NG_", "NM_", "NR_", "NP_")
- `start` (**int**): Start position of the allele  
- `end` (**int**): End position of the allele  
- `allele_state` (**str**): Literal value of the allele sequence state (e.g., ACGT)  
- `id_value` (**str**, optional): The unique identifier for the Allele instance. If not provided, a default ID will be generated in the format 'ref-to-{context_sequence_id}'

#### **`build_vrs_allele()` – Generates a VRS Allele**  

This function constructs a **VRS Allele** using the following attributes:  
- `context_sequence_id` (**str**): Accession number of the reference sequence. Supported prefixes include: ("NC_", "NG_", "NM_", "NR_", "NP_")
- `start` (**int**): Start position of the sallele  
- `end` (**int**): End position of the allele  
- `allele_state` (**str**): Literal value of the allele sequence state (e.g., ACGT)  
- `normalize` (**bool**, default=`True`): Option to normalize the VRS object  

### What This Notebook Demonstrates

This notebook outlines a structured **workflow** to:

1. **Set Up & Import Modules**  
   - Load the `AlleleBuilder` and `VrsFhirAlleleTranslator` modules.

2. **Generate VRS and Translate to FHIR**  
   - Build a **VRS Allele object** and convert it from **VRS → FHIR**.

3. **Round-Trip Translation: VRS → FHIR → VRS**  
   - Perform a **round-trip translation** back to VRS (**VRS → FHIR → VRS**).
   
4. **Generate FHIR and Translate to VRS**  
   - Build a **FHIR Allele object** and convert it from **FHIR → VRS**.

5. **Round-Trip Translation: FHIR → VRS → FHIR**  
   - Perform a **round-trip translation** back to VRS (**FHIR → VRS → FHIR**).

### Set up and import modules

In [ ]:
# Importing the `AlleleBuilder` class from the utils module
from builders.allele_builder import AlleleBuilder
# Importing the `VrsFhirAlleleTranslator` class from the `translators` module
from translators.vrs_fhir_translator import VrsFhirAlleleTranslator
from ga4gh.vrs.dataproxy import create_dataproxy


In [ ]:
dp = create_dataproxy(uri="seqrepo+file:///usr/local/share/seqrepo/2024-12-20")

# Creating an instance of `AlleleFactory` to generate FHIR and VRS Allele objects
build_allele = AlleleBuilder(dp=dp)
# Creating an instance of `VrsFhirAlleleTranslator` to enable bidirectional translation 
# between GA4GH VRS and HL7 FHIR Allele representations
allele_translator= VrsFhirAlleleTranslator(dp=dp)

### Create VRS, translate to FHIR

In [ ]:
# Creating a GA4GH VRS Allele (Version 2.0) using the `create_vrs_allele` function
example_vrs_allele = build_allele.build_vrs_allele(
    context_sequence_id="NC_000002.12",
    start=27453448,
    end=27453449,
    allele_state="T",
    normalize=True
)

# Converting the VRS Allele object into a dictionary representation for easy viewing
example_vrs_allele.model_dump(exclude_none=True)

In [ ]:
# Translating a GA4GH VRS Allele into an HL7 FHIR Allele
# This function takes a VRS Allele object and converts it into its corresponding FHIR representation
vrs_to_fhir_translation_example = allele_translator.translate_allele_to_fhir(example_vrs_allele)

# Converting the translated Allele object into a dictionary representation for easy viewing
vrs_to_fhir_translation_example.model_dump()

### Round-Trip Translation: VRS → FHIR → VRS

In [ ]:
# Translate the FHIR Allele profile back to a VRS Allele object
back_to_vrs = allele_translator.translate_allele_to_vrs(vrs_to_fhir_translation_example)

print("Check if the original and round-tripped VRS Allele are identical.")
print(example_vrs_allele == back_to_vrs)

### Create FHIR, translate to VRS

In [ ]:
# Creating an HL7 FHIR Allele using the `create_fhir_allele` function
example_fhir_allele = build_allele.build_fhir_allele(
    context_sequence_id="NC_000002.12",
    start=27453448,
    end=27453449,
    allele_state="T",
)

# Converting the Allele object into a dictionary representation for easy viewing
example_fhir_allele.model_dump(exclude_none=True)

In [ ]:
# Translating an HL7 FHIR Allele into a GA4GH VRS Allele
# This function converts a FHIR Allele object into its corresponding VRS representation
fhir_to_vrs_translation_example = allele_translator.translate_allele_to_vrs(example_fhir_allele)

# Printing the type of the translated object to confirm the output class
print(type(fhir_to_vrs_translation_example))

# Converting the translated VRS Allele object into a dictionary representation for easy viewing
fhir_to_vrs_translation_example.model_dump(exclude_none=True)

### Round-Trip Translation: FHIR → VRS → FHIR
- The `create_fhir_allele()` function supports round-trip compatibility between FHIR and VRS.
- If you want to enable full round-trip compatibility, **do not provide a custom `id_value`** when constructing an AlleleProfile.
- When `id_value` is omitted, a default identifier is automatically generated in the format:  
  `ref-to-{context_sequence_id}`
- This approach ensures consistent and lossless translation from FHIR → VRS → FHIR.

In [ ]:
# Translate the VRS Allele object back to FHIR Allele 
back_to_fhir = allele_translator.translate_allele_to_fhir(fhir_to_vrs_translation_example)

print("Check if the original and round-tripped FHIR Allele are identical.")
print(example_fhir_allele == back_to_fhir)

### Conclusion

For a more detailed exploration of the implementation, refer to the other notebooks that provide an in-depth, step-by-step guide on creating these objects and performing translations between FHIR and VRS.  